In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import tensorflow as tf
import numpy as np
# from tensorflow_model_optimization.python.core.keras.compat import keras
from tensorflow import keras
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)
from sklearn.metrics import f1_score

# import sys 
# sys.path.append("C:/Users/polin/TinyBirdSounds") 

from create_spectrogram_and_time_series_dataset_use import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

2024-06-20 10:53:04.705335: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 10:53:04.771490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 10:53:04.771588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 10:53:04.777610: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 10:53:04.797012: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 10:53:04.798419: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
sample_rate = 16000

In [4]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-06-19 17:39:03.317522: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-19 17:39:03.317948: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-19 17:39:03.386677: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-19 17:39:03.391174: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [5]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [6]:
# for i in x_val_np:
#     print(i.shape)

In [7]:
BASE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz_baseline.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 182, 78, 4)        40        
                                                                 
 max_pooling2d (MaxPooling2  (None, 91, 39, 4)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 89, 37, 4)         148       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 44, 18, 4)         0         
 g2D)                                                            
                                                                 
 reshape (Reshape)           (None, 3168)              0         
                                                                 
 dense (Dense)               (None, 8)                 2

In [8]:
print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(baseline_model, x_test_np, y_test_np, model_format="keras")


Test dataset:
44/44 [==============================] - 1s 10ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9817187973649888


### Convert to TF Lite format

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmp4t162s7d/assets


INFO:tensorflow:Assets written to: /tmp/tmp4t162s7d/assets
2024-06-18 21:17:02.111870: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 21:17:02.112373: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 21:17:02.114696: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4t162s7d
2024-06-18 21:17:02.115715: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 21:17:02.115727: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4t162s7d
2024-06-18 21:17:02.121602: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-18 21:17:02.122430: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 21:17:02.160925: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [10]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.06%
Recall: 99.36%
Precision: 97.91%
F1-score: 98.63%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.986689172446992

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9861205880566085

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9836996874048178

Time for Test dataset:
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.007 seconds
M

### Dynamic range quantization

In [11]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpvm4ldvlu/assets


INFO:tensorflow:Assets written to: /tmp/tmpvm4ldvlu/assets
2024-06-18 21:17:43.528355: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 21:17:43.528418: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 21:17:43.528788: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpvm4ldvlu
2024-06-18 21:17:43.530210: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 21:17:43.530224: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpvm4ldvlu
2024-06-18 21:17:43.534204: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 21:17:43.579480: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpvm4ldvlu
2024-06-18 21:17:43.595013: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [12]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.06%
Recall: 99.36%
Precision: 97.91%
F1-score: 98.63%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.986689172446992

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.986917253178175

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9842259233992772

Time for Test dataset:
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.005 seconds
Mi

### Float16 quantization

In [ ]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

In [14]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16KHz_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.06%
Recall: 99.36%
Precision: 97.91%
F1-score: 98.63%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.986689172446992

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9817247033115624

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9835029887214596

Time for Test dataset:
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.007 seconds
M

### Float Fallback quantization

In [17]:
float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpnfs8wuz1/assets


INFO:tensorflow:Assets written to: /tmp/tmpnfs8wuz1/assets
/home/polina/test/venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-18 21:19:41.470264: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 21:19:41.470421: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 21:19:41.470610: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpnfs8wuz1
2024-06-18 21:19:41.471922: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 21:19:41.471962: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpnfs8wuz1
2024-06-18 21:19:41.476673: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 21:19:41.

In [19]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz_float_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.13%
Recall: 98.73%
Precision: 98.73%
F1-score: 98.73%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9875259328054573

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9880446659709459

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.64%
Recall: 97.60%
Precision: 98.24%
F1-score: 97.92%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9786514234791476

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9797066234162832

Time for Test dataset:
Accuracy: 98.64%
Recall: 97.60%
Precision: 98.24%
F1-score: 97.92%

Time to make a prediction for a single data point
Mean: 0.001 seconds
Max: 0.004 seconds
